In [ ]:
import numpy as np
from matplotlib import pyplot
import random 

Grid = None # Maze Grid storage

DISTANCE = 16 # length of the square, go for 8-20

Rooms = [] # stores array of rooms

# Creates a grid of x squares by y squares with separators (output is x*DISTANCE + 2 by y*DISTANCE + 2)
# Also used to reset the grid to an initial point
def makeGrid(x : int = 5, y : int = 5):
    if type(x) != type(0):
        raise BaseException('Grid.makeGrid - The input "x" must be an int.')
    if type(y) != type(0):
        raise BaseException('Grid.makeGrid - The input "y" must be an int.')
    grid = []
    for i in range(y * DISTANCE + 2):
        sample = []
        for j in range(x * DISTANCE + 2):
            if i % DISTANCE == 0 or i % DISTANCE == 1 or j % DISTANCE == 0 or j % DISTANCE == 1:
                sample.append((0,0,0)) # black border/grid
            else:
                sample.append((225,225,225)) # white rest as default
        grid.append(sample)
    global Grid
    Grid = np.array(grid, dtype = 'uint8')

def getGrid():
    return Grid
def setGrid(grid):
    global Grid
    Grid = grid

# Could move the below code to another script

# Class for Rooms
class Room:
    # initializes room
    def __init__(self, x, y, width, height, color=(0, 0, 0)):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = color  
    # places generated rooms onto grid
    def place(self, grid):
        max_y, max_x = grid.shape[0], grid.shape[1]  # get grid dimensions
        for i in range(self.y * DISTANCE + 2, (self.y + self.height) * DISTANCE + 2):
            for j in range(self.x * DISTANCE + 2, (self.x + self.width) * DISTANCE + 2):
                if 0 <= i < max_y and 0 <= j < max_x:  #  ensure within bounds
                    if grid[i][j].tolist() != [0, 0, 0]:  # avoid overwriting borders
                        grid[i][j] = self.color
        return grid
    # adds rooms to array
    @staticmethod
    def addRoom(room):
        global Grid, Rooms
        Rooms.append(room)
        Grid = room.place(Grid)       

# Generate random rooms
def generateRandomRooms(count, grid_width, grid_height, max_room_size=5):
    for _ in range(count):
        width = random.randint(2, max_room_size)
        height = random.randint(2, max_room_size)
        x = random.randint(0, grid_width - width)
        y = random.randint(0, grid_height - height)
        color = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
        room = Room(x, y, width, height, color)
        Room.addRoom(room)

   
# Displays the grid using the specified approach, Graph or Text
def displayGrid(variant = 'Graph'):
    return variant == 'Graph' and graph() or variant == 'Text' and text() or None

# Displays the grid as an image
def graph():
    fig = pyplot.figure()
    ax = pyplot.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(Grid)
    pass

# for later, just using it to test image quality for now
def save():
    pyplot.imsave('sample.png', Grid)

def text():
    # ...
    pass

In [ ]:
# Test, using ipynb to make it easier to see the results in Github without having to run it yourself.  Will probably change to .py later.
makeGrid(x = 30, y = 30)
#Test for adding a number of rooms, in a 
generateRandomRooms(count=8, grid_width=30, grid_height=30, max_room_size=8)  # 10 random rooms

displayGrid()

In [ ]:
import numpy as np

# Helper function to print and run test cases with debugging output
def run_test(test_func):
    print(f"\n[TEST] {test_func.__name__}: {test_func.__doc__}")  # Print test description
    try:
        test_func()
        print(f"[PASS] {test_func.__name__}")
    except AssertionError as e:
        print(f"[FAIL] {test_func.__name__} - {e}")
    except Exception as e:
        print(f"[ERROR] {test_func.__name__} - {e}")

# Test Case 1: Grid Initialization with Valid Inputs
def test_makeGrid_valid():
    """Ensures the grid is correctly initialized with expected dimensions and border colors."""
    x, y = 5, 5
    print(f"Creating grid with dimensions: {x}x{y} (Cells: {(x * DISTANCE + 2)}x{(y * DISTANCE + 2)})")
    makeGrid(x, y)
    grid = getGrid()
    
    print(f"Checking grid shape: Expected {(x * DISTANCE + 2, y * DISTANCE + 2, 3)}, Got {grid.shape}")
    assert grid.shape == (y * DISTANCE + 2, x * DISTANCE + 2, 3), "Grid dimensions are incorrect."
    
    print("Checking top-left border cell (should be black):", grid[0, 0])
    assert (grid[0, 0] == [0, 0, 0]).all(), "Grid borders should be black."
    displayGrid()

# Test Case 2: Grid Initialization with Invalid Inputs
def test_makeGrid_invalid():
    """Verifies that invalid inputs raise the correct exceptions."""
    invalid_inputs = [("5", 5), (5, "5")]
    
    for x, y in invalid_inputs:
        print(f"Testing invalid input: x={x}, y={y}")
        try:
            makeGrid(x, y)
            assert False, f"Expected exception for x={x}, y={y}"
        except BaseException as e:
            print(f"Correctly caught exception: {e}")
    displayGrid()

# Test Case 3: Room Placement Within Bounds
def test_roomPlacement():
    """Checks if a room is correctly placed inside the grid boundaries."""
    makeGrid(10, 10)
    room = Room(1, 1, 3, 3, (100, 100, 100))
    print(f"Placing room at x={room.x}, y={room.y}, width={room.width}, height={room.height}, color={room.color}")
    room.place(Grid)
    
    check_x, check_y = (room.y * DISTANCE + 3, room.x * DISTANCE + 3)
    print(f"Checking room color at ({check_y}, {check_x}): Expected {room.color}, Got {Grid[check_y, check_x]}")
    assert (Grid[check_y, check_x] == room.color).all(), "Room color not applied correctly."
    displayGrid()

# Test Case 4: Prevent Room Overlapping Borders
def test_roomBorders():
    """Ensures that rooms do not overwrite the black grid borders."""
    makeGrid(10, 10)
    room = Room(0, 0, 3, 3, (100, 100, 100))
    print(f"Placing room at the top-left corner: {room.x}, {room.y}, should not overwrite borders.")
    room.place(Grid)

    print("Checking border cell at (0,0): Expected black (0,0,0), Got", Grid[0, 0])
    assert (Grid[0, 0] == [0, 0, 0]).all(), "Room should not overwrite grid borders."
    displayGrid()

# Test Case 5: Generate Random Rooms and Verify Grid Change
def test_generateRandomRooms():
    """Ensures that generating random rooms modifies the grid as expected."""
    makeGrid(10, 10)
    initial_grid = Grid.copy()
    
    print("Generating 5 random rooms...")
    generateRandomRooms(5, 10, 10)

    print("Checking if grid has changed after adding rooms...")
    assert not np.array_equal(initial_grid, Grid), "Grid did not change after adding rooms."
    displayGrid()

# Test Case 6: Validate Graph Display
def test_displayGraph():
    """Verifies that the graph display function does not return any value."""
    makeGrid(5, 5)
    print("Displaying grid as graph (should see a visual output)")
    assert displayGrid('Graph') is None, "Graph function should not return anything."

# Test Case 7: Validate Text Display (Placeholder)
def test_displayText():
    """Checks that the text display function is correctly handled (currently not implemented)."""
    print("Checking text display output (should return None since it's not implemented)")
    assert displayGrid('Text') is None, "Text display is not implemented yet, should return None."

# Run all tests
test_cases = [
    test_makeGrid_valid,
    test_makeGrid_invalid,
    test_roomPlacement,
    test_roomBorders,
    test_generateRandomRooms,
    test_displayGraph,
    test_displayText,
]

print("Running tests...\n")
for test in test_cases:
    run_test(test)

print("\nAll tests completed!")